# Create daughter plate

In [1]:
import sys
sys.path.append('/home/apaulson/repos/data-proc/')
import mysql_cnx as mc
%config Completer.use_jedi=False

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import text, select, or_, and_

In [72]:
engine.dispose()

In [7]:
# engine=mc.create_mysql_cnx(db='QB3_HiTS_nightly')

engine=mc.create_mysql_cnx(db='QB3_HiTS')

with engine.connect() as conn:
    result=conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


In [8]:
# MetaData stores a collection of tables we want to work with
from sqlalchemy import MetaData
metadata = MetaData()

In [9]:
# reflection gets an existing table from the db
from sqlalchemy import Table

inv_plate=Table("INV_PLATE", metadata, autoload_with=engine)
inv_well=Table("INV_WELL", metadata, autoload_with=engine)
inv_comp_lot=Table("INV_COMP_LOT", metadata, autoload_with=engine)

In [10]:
inv_plate_cols=[x.name for x in inv_plate.columns]
inv_plate_cols=[x for x in inv_plate_cols if x != inv_plate.primary_key.columns[0].name]

inv_well_cols=[x.name for x in inv_well.columns]
inv_well_cols=[x for x in inv_well_cols if x != inv_well.primary_key.columns[0].name]

In [11]:
# insert
from sqlalchemy import insert
from datetime import datetime

In [12]:
def stringnow():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [13]:
inv_plate.primary_key.columns[0].name

'IP_ID'

## lookup plate and well info to copy here

In [35]:
gridnums=pd.read_csv('/Users/apaulson/Downloads/daughter_plates.csv')
# barcodes=barcodes.Grid_num.tolist()

In [36]:
# gridnums=pd.DataFrame(barcodes, columns=['barcodes'])
# print(gridnums.shape)
# gridnums['grid_num']=gridnums.barcodes.str.strip('-Eco1')
gridnums.barcodes=gridnums.barcodes.str.strip()
gridnums.shape

(7, 3)

In [37]:
gridnums=gridnums.drop_duplicates()
gridnums.shape

(7, 3)

In [38]:
# check for existing barcodes
query=select(inv_plate.c.IP_BARCODE).where(inv_plate.c.IP_BARCODE.in_(gridnums.barcodes.tolist()))
existing_barcodes=[]
with engine.connect() as conn:
    for row in conn.execute(query):
        existing_barcodes.append(row[0])
len(existing_barcodes)

0

In [39]:
# remove if so
gridnums=gridnums[~gridnums.barcodes.isin(existing_barcodes)]
gridnums.shape

(7, 3)

In [43]:
# double check plate type, should be 1 if possible
clauses = [inv_plate.c.IP_GRID_NUM.like(elem) for elem in gridnums.grid_num]
query = select(inv_plate).filter(and_(
    inv_plate.c.IP_IPT_ID==6, 
    or_(*clauses)))
source_plates=pd.read_sql(query, con=engine)
source_plates=source_plates[source_plates.IP_BARCODE.str.contains('PT01')]
source_plates.shape

(7, 16)

In [44]:
query = select(inv_well).where(inv_well.c.IW_IP_ID.in_(source_plates.IP_ID.tolist()))
source_wells=pd.read_sql(query, con=engine)
print(len(source_wells)/len(source_plates))
source_wells.head(2)

384.0


,IW_ID,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES
0,7790187,6877163,24824,None,1,A01,NaN,NaN,NaN,NaN,None,None,107,2019-07-10 11:36:38,NaN,None
1,7790188,6877164,24824,None,3,A03,1090090.0,5.0,50000.0,NaN,None,None,107,2019-07-10 11:36:38,NaN,conc updated to 50mM 20230824 by AKP


In [45]:
source_plate_dict=dict(zip(source_plates.IP_BARCODE, source_plates.IP_ID))
source_well_dict=dict(zip(source_wells.IW_ID, source_wells.IW_IP_ID.astype(str)+'_'+source_wells.IW_COORDINATES))
dest_plate_dict=dict(zip(gridnums.grid_num, gridnums.barcodes))
source_plate_dict

{'SMDC-MPHORE-384-1-02-PT01': 25708,
 'SMDC-MPHORE-384-2-02-PT01': 25711,
 'SMDC-MPHORE-384-3-02-PT01': 25714,
 'SMDC-MPHORE-384-4-02-PT01': 25717,
 'SMDC-MPHORE-384-5-02-PT01': 25720,
 'SMDC-MPHORE-384-6-01-PT01': 24824,
 'SMDC-MPHORE-384-7-01-PT01': 27235}

In [48]:
plates=source_plates.copy()
plates.IP_SOURCE_ID=plates.IP_BARCODE.map(source_plate_dict)
plates=plates.drop(columns='IP_ID')
plates.IP_IPT_ID=9 # 9 is acoustic dispensing plates
plates.IP_BARCODE=plates.IP_GRID_NUM.map(dest_plate_dict)
plates.IP_INS_BY=569
plates.IP_INS_DATE=stringnow()

plates.IP_NOTES=gridnums.notes.tolist()
plates

,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER
0,25708,SMDC-MPHORE-384-1-02,9,2,None,SMDC-MPHORE-384-1-02-Eco1,1,None,None,148,SMDC Monophore,05342DS,569,2023-10-13 19:40:58,SMDC
3,25711,SMDC-MPHORE-384-2-02,9,2,None,SMDC-MPHORE-384-2-02-Eco1,1,None,None,148,SMDC Monophore,05343DS,569,2023-10-13 19:40:58,SMDC
6,25714,SMDC-MPHORE-384-3-02,9,2,None,SMDC-MPHORE-384-3-02-Eco1,1,None,None,148,SMDC Monophore,05344DS,569,2023-10-13 19:40:58,SMDC
9,25717,SMDC-MPHORE-384-4-02,9,2,None,SMDC-MPHORE-384-4-02-Eco1,1,None,None,148,SMDC Monophore,05345DS,569,2023-10-13 19:40:58,SMDC
12,25720,SMDC-MPHORE-384-5-02,9,2,None,SMDC-MPHORE-384-5-02-Eco1,1,None,None,148,SMDC Monophore,05346DS,569,2023-10-13 19:40:58,SMDC
15,24824,SMDC-MPHORE-384-6-01,9,2,None,SMDC-MPHORE-384-6-02-Eco1,1,None,None,148,SMDC Monophore,05347DS,569,2023-10-13 19:40:58,SMDC
17,27235,SMDC-MPHORE-384-7-01,9,2,None,SMDC-MPHORE-384-7-02-Eco1,1,None,None,148,SMDC Monophore,05348DS,569,2023-10-13 19:40:58,SMDC


In [49]:
# check for important plate cols
assert set(plates.columns)==set(inv_plate_cols), "Missing info for INV_PLATE table"
assert len(plates[plates.IP_GRID_NUM.isna()])==0, "Missing GridNum"
assert len(plates[plates.IP_BARCODE.isna()])==0, "Missing Barcode"
assert len(plates[plates.IP_GRID_NUM.duplicated()])==0, "Duplicate gridnums"
assert len(plates[plates.IP_BARCODE.duplicated()])==0, "Duplicate barcodes"

In [50]:
# check existing IP_BARCODES
from sqlalchemy import select

stmt=select(inv_plate).where(inv_plate.c.IP_BARCODE.in_(plates.IP_BARCODE.tolist()))
with engine.connect() as conn:
    result=conn.execute(stmt)
    exists=result.rowcount>0

if exists:
    for i, row in enumerate(result):
        print(row)
        ip_id=row[0]
        assert i==0, "Warning: this barcode already exists"
else:
    # stmt=insert(inv_plate).values(plate)    
    # compiled=stmt.compile()
    # with engine.connect() as conn:
    #     result=conn.execute(stmt)
    #     conn.commit()
    # ip_id=result.inserted_primary_key
    ip_id=np.nan
    print('OK')

OK


In [51]:
ip_id

nan

In [52]:
import sqlalchemy
if isinstance(ip_id,sqlalchemy.engine.row.Row):
    ip_id=ip_id[0]
ip_id

nan

In [53]:
# # insert.. careful!!
# plates.to_sql(name='INV_PLATE', con=engine, if_exists = 'append', index=False)

7

In [54]:
# get IP_IDs
ip_ids=pd.read_sql(stmt, con=engine)
ip_ids

,IP_ID,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER
0,27862,25708,SMDC-MPHORE-384-1-02,9,2,None,SMDC-MPHORE-384-1-02-Eco1,1,None,None,148,SMDC Monophore,05342DS,569,2023-10-13 19:40:58,SMDC
1,27863,25711,SMDC-MPHORE-384-2-02,9,2,None,SMDC-MPHORE-384-2-02-Eco1,1,None,None,148,SMDC Monophore,05343DS,569,2023-10-13 19:40:58,SMDC
2,27864,25714,SMDC-MPHORE-384-3-02,9,2,None,SMDC-MPHORE-384-3-02-Eco1,1,None,None,148,SMDC Monophore,05344DS,569,2023-10-13 19:40:58,SMDC
3,27865,25717,SMDC-MPHORE-384-4-02,9,2,None,SMDC-MPHORE-384-4-02-Eco1,1,None,None,148,SMDC Monophore,05345DS,569,2023-10-13 19:40:58,SMDC
4,27866,25720,SMDC-MPHORE-384-5-02,9,2,None,SMDC-MPHORE-384-5-02-Eco1,1,None,None,148,SMDC Monophore,05346DS,569,2023-10-13 19:40:58,SMDC
5,27867,24824,SMDC-MPHORE-384-6-01,9,2,None,SMDC-MPHORE-384-6-02-Eco1,1,None,None,148,SMDC Monophore,05347DS,569,2023-10-13 19:40:58,SMDC
6,27868,27235,SMDC-MPHORE-384-7-01,9,2,None,SMDC-MPHORE-384-7-02-Eco1,1,None,None,148,SMDC Monophore,05348DS,569,2023-10-13 19:40:58,SMDC


## edit well info here via plate map file

In [55]:
# wells
replace_ipids=dict(zip(ip_ids.IP_SOURCE_ID, ip_ids.IP_ID))

In [56]:
wells=source_wells.copy()
wells.IW_SOURCE_ID=wells.IW_ID
wells=wells.drop(columns='IW_ID')
wells.IW_IP_ID=wells.IW_IP_ID.map(replace_ipids)
wells.IW_INS_BY=569
wells.IW_INS_DATE=stringnow()
wells.head(2)

,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES
0,7790187,27867,None,1,A01,NaN,NaN,NaN,NaN,None,None,569,2023-10-13 19:42:04,NaN,None
1,7790188,27867,None,3,A03,1090090.0,5.0,50000.0,NaN,None,None,569,2023-10-13 19:42:04,NaN,conc updated to 50mM 20230824 by AKP


### add volume and concentration here

In [57]:
wells.IW_CONC_UM.unique()

array([   nan, 50000.])

In [58]:
vol_ul=5
conc_um=50000

In [59]:
wells.loc[~wells.IW_ICL_ID.isna(), 'IW_VOL_UL']=vol_ul
wells.loc[~wells.IW_ICL_ID.isna(), 'IW_CONC_UM']=conc_um
wells

,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES
0,7790187,27867,None,1,A01,NaN,NaN,NaN,NaN,None,None,569,2023-10-13 19:42:04,NaN,None
1,7790188,27867,None,3,A03,1090090.0,5.0,50000.0,NaN,None,None,569,2023-10-13 19:42:04,NaN,conc updated to 50mM 20230824 by AKP
2,7790189,27867,None,5,A05,1089999.0,5.0,50000.0,NaN,None,None,569,2023-10-13 19:42:04,NaN,conc updated to 50mM 20230824 by AKP
3,7790190,27867,None,7,A07,1090010.0,5.0,50000.0,NaN,None,None,569,2023-10-13 19:42:04,NaN,conc updated to 50mM 20230824 by AKP
4,7790191,27867,None,9,A09,1090008.0,5.0,50000.0,NaN,None,None,569,2023-10-13 19:42:04,NaN,conc updated to 50mM 20230824 by AKP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,8371540,27868,None,145,G01,NaN,NaN,NaN,NaN,None,None,569,2023-10-13 19:42:04,0.0,None
2684,8371541,27868,None,315,N03,NaN,NaN,NaN,NaN,None,None,569,2023-10-13 19:42:04,0.0,None
2685,8371542,27868,None,343,O07,1186360.0,5.0,50000.0,50000000.0,None,None,569,2023-10-13 19:42:04,0.0,conc updated to 50mM 20230824 by AKP
2686,8371543,27868,None,380,P20,NaN,NaN,NaN,NaN,None,None,569,2023-10-13 19:42:04,0.0,None


In [60]:
# check IP_ID doesn't exist in INV_WELL yet
stmt=select(inv_well).where(inv_well.c.IW_IP_ID.in_(tuple(wells.IW_IP_ID.unique())))
with engine.connect() as conn:
    result=conn.execute(stmt)
    assert result.rowcount==0, "Wells exist for these plates"

In [61]:
wells.shape, len(wells)/384

((2688, 15), 7.0)

In [62]:
# # insert... careful here
# wells.to_sql(name='INV_WELL', con=engine, if_exists = 'append', index=False)

2688

In [63]:
engine.dispose()

In [64]:
existing_barcodes

[]

In [70]:
gridnums.barcodes.tolist()

['SMDC-MPHORE-384-1-02-Eco1',
 'SMDC-MPHORE-384-2-02-Eco1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
 'SMDC-MPHORE-384-3-02-Eco1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
 'SMDC-MPHORE-384-4-02-Eco1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
 'SMDC-MPHORE-384-5-02-Eco1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
 'SMDC-MPHORE-384-6-02-Eco1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0',
 'SMDC-MPHORE-384-7-02-Eco1\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0']

In [71]:
gridnums.barcodes=gridnums.barcodes.str.strip()
gridnums.barcodes.tolist()

['SMDC-MPHORE-384-1-02-Eco1',
 'SMDC-MPHORE-384-2-02-Eco1',
 'SMDC-MPHORE-384-3-02-Eco1',
 'SMDC-MPHORE-384-4-02-Eco1',
 'SMDC-MPHORE-384-5-02-Eco1',
 'SMDC-MPHORE-384-6-02-Eco1',
 'SMDC-MPHORE-384-7-02-Eco1']